# SETUP

In [ ]:
;;
#require "pkp"

In [ ]:
open Printf

open Pkp

open Single_neuron

In [ ]:
let plot_signal x =
  let plot (module P : Plot) = P.plot (A x) ~style:"l lc 8 lw 2" [barebone] in
  Juplot.draw ~fmt:`svg ~size:(800, 200) plot

In [ ]:
(* data: array of (time, state) arrays *)
let plot_voltage ?colors ?(voltage_range = (-100., 40.)) data =
  let open Gp in
  let plot (module P : Plot) =
    let props =
      [ barebone
      ; borders [`left]
      ; yrange voltage_range
      ; ytics (`regular [-100.; 40.]) ~o:"out nomirror"
      ; set "offsets graph 0.02, 0, 0, 0"
      ; ylabel "V_m [mV]" ~o:"offset 4, 0 norotate" ]
    in
    let items =
      Array.mapi
        (fun i (t, u) ->
          let color = match colors with Some z -> z.(i) | None -> "black" in
          item (L [t; Mat.col u 0]) ~style:(sprintf "l lc rgb '%s' lw 2" color)
          )
        data
      |> Array.to_list
    in
    P.plots items props
  in
  Juplot.draw ~fmt:`svg ~size:(500, 120) plot

# NOBEL PRIZE-WINNING MODEL (HODGKIN-HUXLEY, 1952)

In [ ]:
module H = HH (HH_default_prms)

## Simulation with constant input current

Here we can play with the input current and see that the neuron only spikes for large enough current:

In [ ]:
let _ =
  let input t = if t < 0.02 then 0. else 1.5E-3 in
  let t, u = H.simulate ~duration:0.2 input in
  plot_voltage ~voltage_range:(-82., 50.) [|(t, u)|]

Now, let's plot the voltage for various amplitudes of the step current: 

In [ ]:
let _ =
  [|1.2; 2.; 4.|]
  |> Array.map (fun c ->
         H.simulate ~duration:0.1 (fun t -> if t < 0.01 then 0. else c *. 1E-3)
     )
  |> plot_voltage ~voltage_range:(-82., 50.)
       ~colors:[|"#1b9e77"; "#d95f02"; "#7570b3"|]

Let's inspect the "gating variables": can you guess which one is which?

In [ ]:
let _ =
  let t, u =
    H.simulate ~duration:0.1 (fun t -> if t < 0.02 then 0. else 4E-3)
  in
  let open Gp in
  let plot (module P : Plot) =
    let p0 =
      [ barebone
      ; borders [`left]
      ; margins [`left 0.3; `right 0.9]
      ; xrange (0.015, 0.04) ]
    in
    let p1 =
      [ margins [`top 0.9; `bottom 0.6]
      ; ylabel "V_m [mV]" ~o:"offset 3, 0 norotate"
      ; ytics (`regular [-70.; 20.]) ~o:"out nomirror" ]
    in
    let p2 =
      [ margins [`top 0.5; `bottom 0.2]
      ; ylabel "gates"
      ; yrange (0., 1.01)
      ; ytics (`regular [0.; 1.0]) ~o:"out nomirror" ]
    in
    (* plot voltage at the top *)
    P.plot (L [t; Mat.col u 0]) ~style:"l lc 8 lw 2" (p0 @ p1) ;
    (* plot the gating variables *)
    P.plots
      (List.init 3 (fun i ->
           let i = i + 1 in
           item (L [t; Mat.col u i]) ~style:(sprintf "l lc %i lw 2" i) ))
      (p0 @ p2)
  in
  Juplot.draw ~fmt:`svg ~size:(500, 300) plot

## Simulation with random input current

In [ ]:
let _ = plot_signal Misc.(ou_process ~tau:20E-3 ~dt:1E-3 ~duration:1.0)

In [ ]:
let _ =
  let noise =
    let dt = 1E-3 in
    let x = Misc.ou_process ~tau:20E-3 ~dt ~duration:1.1 in
    fun t -> Mat.get x 0 (int_of_float (t /. dt))
  in
  let t, u = H.simulate ~duration:1.0 (fun t -> 0.7E-3 *. (0. +. noise t)) in
  plot_voltage ~yrange:(-110., 30.) [|(t, u)|]

----

# A SIMPLER MODEL ("LEAKY INTEGRATE-AND-FIRE")

In [ ]:
module L = LIF (LIF_default_prms)

In [ ]:
let _ =
  [|2.0; 5.0; 15.0; 25.0|]
  |> Array.map (fun c ->
         L.simulate ~duration:0.1 (fun t -> if t < 0.02 then 0. else c) )
  |> plot_voltage ~yrange:(-75., 2.)
       ~colors:[|"#1b9e77"; "#d95f02"; "#7570b3"; "#e7298a"; "#66a61e"|]

In [ ]:
let _ =
  let duration = 1. in
  let dt = 1E-3 in
  let noise =
    let x = Misc.ou_process ~tau:20E-3 ~dt ~duration:(1.1 *. duration) in
    fun t -> Mat.get x 0 (int_of_float (t /. dt))
  in
  let t, u = L.simulate ~duration (fun t -> 5. *. noise t) in
  plot_voltage ~yrange:(-85., 2.) [|(t, u)|]